In [ ]:
# This example is modified from

# https://huggingface.co/gpt2
# https://huggingface.co/blog/how-to-generate

In [ ]:
# In google colab, make sure you install transformers
# uncomment the following line for first-time execution
#!pip install transformers

In [ ]:
from time import time

In [2]:
import torch
from transformers import pipeline, set_seed
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel



```
# Load Tokenizer and Model
```



In [3]:
# you need to download gpt2 by yourself through vpn, if you run this locally
# in colab, this works well
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = TFGPT2LMHeadModel.from_pretrained('gpt2')

OSError: Can't load tokenizer for 'gpt2'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'gpt2' is the correct path to a directory containing all relevant files for a GPT2Tokenizer tokenizer.

In [ ]:
text = "Name a good tennis player."

# Method 1: Use Transformers Pipeline to direct generate new sentences

In [ ]:
set_seed(42)
generator = pipeline('text-generation', model='gpt2')
generator(text, max_length=100, num_return_sequences=5)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Replace me by any text you\'d like. You\'re going to look like the guy who beat the drums in the \'40s?" I tried to keep a small smile.\n\n\nCrazy as I thought that, he said, "I wish I could meet you tomorrow, just to tell you what I\'m thinking.""Hey, I just wanted to get home and see him. I just tried to play him the music while he was playing. He played like a lot of people, and'},
 {'generated_text': 'Replace me by any text you\'d like. Please leave it with any comments for this post and use #freenode for all comments.\n\nThis is the #freenode "rpc". It is a lightweight, low end fork of the Linux RPC protocol that allows users to implement RPC commands without needing to write or debug a lot of code. This fork also makes it easier for people to interact with the rpc on a wide variety of devices to make use of it without breaking'},
 {'generated_text': "Replace me by any text you'd like.\n\nYou can also try to play the game through SMS or a regular browser (I ju

Method 2: We let the model generate (forward) and tokenize back to words

In [ ]:


# tokenize input prompt
encoded_input = tokenizer(text, return_tensors='tf')
print(encoded_input)


# check https://github.com/huggingface/transformers/blob/main/src/transformers/generation/configuration_utils.py#L40
generation_kwargs = {
    "min_length": -1,
    "top_k": 0.0,
    "top_p": 1.0,
    "do_sample": True,
    "pad_token_id": tokenizer.eos_token_id,
    "max_new_tokens": 16,
    "num_return_sequences":10,
}

{'input_ids': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=
array([[3041, 5372,  502,  416,  597, 2420,  345, 1549,  588,   13]],
      dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 10), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}


In [ ]:

# check KV cache option
# https://github.com/huggingface/transformers/blob/main/src/transformers/generation/configuration_utils.py#L100

t1 = time()
set_seed(42)
output_tensors_1 = model.generate(encoded_input['input_ids'], use_cache=True, **generation_kwargs)
#print(output)
t2 = time()
set_seed(42)
output_tensors_2 = model.generate(encoded_input['input_ids'], use_cache=False, **generation_kwargs)
t3= time()

print(t2-t1, t3-t2)

responses_1 = tokenizer.batch_decode(output_tensors_1)
responses_2 = tokenizer.batch_decode(output_tensors_2)

# Compute sentiment score
for response_1, response_2 in zip(responses_1, responses_2):
  print('R1:', response_1, '  ====  ', 'R2:', response_2)
  print('\n\n')


5.3168885707855225 3.1891257762908936
R1: Replace me by any text you'd like.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>   ====   R2: Replace me by any text you'd like.<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>



R1: Replace me by any text you'd like. Like" so each two lines--yes!!!!! The corrolla will become   ====   R2: Replace me by any text you'd like. Like" so each two lines--yes!!!!! The corrolla will become



R1: Replace me by any text you'd like. Checksum : 3F666257B901 Dect asoneTOL   ====   R2: Replace me by any text you'd like. Checksum : 3F666257B901 Dect asoneTOL



R1: Replace me by any text you'd like.<|endoftext|><|endoftext|><|endoftext|><|end